In [11]:
#!/usr/bin/env python3
"""Process an image with the trained neural network
Usage:
    demo.py [options] <yaml-config> <checkpoint> <images>...
    demo.py (-h | --help )

Arguments:
   <yaml-config>                 Path to the yaml hyper-parameter file
   <checkpoint>                  Path to the checkpoint
   <images>                      Path to images

Options:
   -h --help                     Show this screen.
   -d --devices <devices>        Comma seperated GPU devices [default: 0]
"""

import os
import os.path as osp
import pprint
import random
import cv2

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import skimage.io
import skimage.transform
import torch
import yaml
from docopt import docopt

import lcnn
from lcnn.config import C, M
from lcnn.models.line_vectorizer import LineVectorizer
from lcnn.models.multitask_learner import MultitaskHead, MultitaskLearner
from lcnn.postprocess import postprocess
from lcnn.utils import recursive_to

In [2]:
PLTOPTS = {"color": "#33FFFF", "s": 15, "edgecolors": "none", "zorder": 5}
cmap = plt.get_cmap("jet")
norm = mpl.colors.Normalize(vmin=0.9, vmax=1.0)
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])


def c(x):
    return sm.to_rgba(x)

In [7]:
#args = docopt(__doc__)
# config_file = args["<yaml-config>"] or "config/wireframe.yaml"
config_file = "config/wireframe.yaml"
C.update(C.from_yaml(filename=config_file))
M.update(C.model)
#pprint.pprint(C, indent=4)

random.seed(0)
np.random.seed(0)
torch.manual_seed(0)

device_name = "cpu"
# os.environ["CUDA_VISIBLE_DEVICES"] = args["--devices"]
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
if torch.cuda.is_available():
    device_name = "cuda"
    torch.backends.cudnn.deterministic = True
    torch.cuda.manual_seed(0)
    print("Let's use", torch.cuda.device_count(), "GPU(s)!")
else:
    print("CUDA is not available")
device = torch.device(device_name)
# checkpoint = torch.load(args["<checkpoint>"], map_location=device)
checkpoint = torch.load("190418-201834-f8934c6-lr4d10-312k.pth.tar", map_location=device)

# Load model
model = lcnn.models.hg(
    depth=M.depth,
    head=lambda c_in, c_out: MultitaskHead(c_in, c_out),
    num_stacks=M.num_stacks,
    num_blocks=M.num_blocks,
    num_classes=sum(sum(M.head_size, [])),
)
model = MultitaskLearner(model)
model = LineVectorizer(model)
model.load_state_dict(checkpoint["model_state_dict"])
model = model.to(device)
model.eval()
print()

Let's use 1 GPU(s)!



In [39]:
img_dir = "/Users/a1222060/abist/obayashi/data/image/MovieCut/"
out_dir = "/Users/a1222060/abist/obayashi/data/output/MovieCut/moviecut_lcnn/"
imnames = os.listdir(img_dir)

In [40]:
imname = "/Users/a1222060/abist/obayashi/data/image/iPadPro/IMG_0740.jpg"
for name in imnames:
    imname = img_dir + name
    print(f"Processing {imname}")
    im = skimage.io.imread(imname)
    if im.ndim == 2:
        im = np.repeat(im[:, :, None], 3, 2)
    im = im[:, :, :3]
    im_resized = skimage.transform.resize(im, (512, 512)) * 255
    image = (im_resized - M.image.mean) / M.image.stddev
    image = torch.from_numpy(np.rollaxis(image, 2)[None].copy()).float()
    with torch.no_grad():
        input_dict = {
            "image": image.to(device),
            "meta": [
                {
                    "junc": torch.zeros(1, 2).to(device),
                    "jtyp": torch.zeros(1, dtype=torch.uint8).to(device),
                    "Lpos": torch.zeros(2, 2, dtype=torch.uint8).to(device),
                    "Lneg": torch.zeros(2, 2, dtype=torch.uint8).to(device),
                }
            ],
            "target": {
                "jmap": torch.zeros([1, 1, 128, 128]).to(device),
                "joff": torch.zeros([1, 1, 2, 128, 128]).to(device),
            },
            "mode": "testing",
        }
        H = model(input_dict)["preds"]

    lines = H["lines"][0].cpu().numpy() / 128 * im.shape[:2]
    scores = H["score"][0].cpu().numpy()
    for i in range(1, len(lines)):
        if (lines[i] == lines[0]).all():
            lines = lines[:i]
            scores = scores[:i]
            break

    # postprocess lines to remove overlapped lines
    diag = (im.shape[0] ** 2 + im.shape[1] ** 2) ** 0.5
    nlines, nscores = postprocess(lines, scores, diag * 0.01, 0, False)

    t = [0.94, 0.95, 0.96, 0.97, 0.98, 0.99]
    input_data = cv2.imread(imname)
    for l, s in zip(nlines, nscores):
        if s>0.98:
            x1,y1 = map(int, l[0])
            x2,y2 = map(int, l[1])
            img_line = cv2.line(input_data, (y1,x1), (y2,x2),(0,0,255),5)
    #cv2.imwrite("output/moviecut_wireframe/"+img_name, img_line)
    #plt.imshow(img_line)
    cv2.imwrite(out_dir+name, img_line)

Processing /Users/a1222060/abist/obayashi/data/image/MovieCut/frame0.jpg


C:\Users\a1222060\abist\obayashi\Git\lcnn\lcnn\models\line_vectorizer.py:174: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  y = (index // 128).float() + torch.gather(joff[:, 0], 1, index) + 0.5


Processing /Users/a1222060/abist/obayashi/data/image/MovieCut/frame1020.jpg
Processing /Users/a1222060/abist/obayashi/data/image/MovieCut/frame1050.jpg
Processing /Users/a1222060/abist/obayashi/data/image/MovieCut/frame1080.jpg
Processing /Users/a1222060/abist/obayashi/data/image/MovieCut/frame1110.jpg
Processing /Users/a1222060/abist/obayashi/data/image/MovieCut/frame1140.jpg
Processing /Users/a1222060/abist/obayashi/data/image/MovieCut/frame1170.jpg
Processing /Users/a1222060/abist/obayashi/data/image/MovieCut/frame120.jpg
Processing /Users/a1222060/abist/obayashi/data/image/MovieCut/frame1200.jpg
Processing /Users/a1222060/abist/obayashi/data/image/MovieCut/frame1230.jpg
Processing /Users/a1222060/abist/obayashi/data/image/MovieCut/frame1260.jpg
Processing /Users/a1222060/abist/obayashi/data/image/MovieCut/frame1290.jpg
Processing /Users/a1222060/abist/obayashi/data/image/MovieCut/frame1320.jpg
Processing /Users/a1222060/abist/obayashi/data/image/MovieCut/frame1350.jpg
Processing /U

# 閾値の調査

In [44]:
input_dir = "/Users/a1222060/abist/obayashi/data/image/test/"
out_dir = "/Users/a1222060/abist/obayashi/data/output/lcnn/IMG_0750/"
file_name = "IMG_0750.jpg"
imname = input_dir + file_name
#imname = img_dir + name
print(f"Processing {imname}")
im = skimage.io.imread(imname)
if im.ndim == 2:
    im = np.repeat(im[:, :, None], 3, 2)
im = im[:, :, :3]
im_resized = skimage.transform.resize(im, (512, 512)) * 255
image = (im_resized - M.image.mean) / M.image.stddev
image = torch.from_numpy(np.rollaxis(image, 2)[None].copy()).float()
with torch.no_grad():
    input_dict = {
        "image": image.to(device),
        "meta": [
            {
                "junc": torch.zeros(1, 2).to(device),
                "jtyp": torch.zeros(1, dtype=torch.uint8).to(device),
                "Lpos": torch.zeros(2, 2, dtype=torch.uint8).to(device),
                "Lneg": torch.zeros(2, 2, dtype=torch.uint8).to(device),
            }
        ],
        "target": {
            "jmap": torch.zeros([1, 1, 128, 128]).to(device),
            "joff": torch.zeros([1, 1, 2, 128, 128]).to(device),
        },
        "mode": "testing",
    }
    H = model(input_dict)["preds"]

lines = H["lines"][0].cpu().numpy() / 128 * im.shape[:2]
scores = H["score"][0].cpu().numpy()
for i in range(1, len(lines)):
    if (lines[i] == lines[0]).all():
        lines = lines[:i]
        scores = scores[:i]
        break

# postprocess lines to remove overlapped lines
diag = (im.shape[0] ** 2 + im.shape[1] ** 2) ** 0.5
nlines, nscores = postprocess(lines, scores, diag * 0.01, 0, False)

#t = [0.94, 0.95, 0.96, 0.97, 0.98, 0.99, 1.00]
threshold = []
for i in range(10):
    threshold.append(round(1.0-0.01*i, 2))
    
for i in range(1, 6):
    threshold.append(round(1.0-0.1*i, 2))

input_data = cv2.imread(imname)

for t in threshold:
    for l, s in zip(nlines, nscores):
        if s>t:
            x1,y1 = map(int, l[0])
            x2,y2 = map(int, l[1])
            img_line = cv2.line(input_data, (y1,x1), (y2,x2),(0,0,255),5)
    #cv2.imwrite("output/moviecut_wireframe/"+img_name, img_line)
    #plt.imshow(img_line)
    cv2.imwrite(out_dir+str(t)+"_"+file_name, img_line)

Processing /Users/a1222060/abist/obayashi/data/image/test/IMG_0750.jpg
